In [1]:
import os
import glob
import pandas as pd
import csv
import requests
import time
import random
from bs4 import BeautifulSoup

In [2]:
seed_path = os.path.join('data', 'seed')
top_songs_filename = os.path.join('data', 'raw', 'top_songs_with_lyrics.csv')

# Read data

In [3]:
all_files = glob.glob(seed_path + "/*.csv")
song_data = pd.concat(pd.read_csv(f) for f in all_files)
song_data.head()

,rank,artists,title
0,1,Maluma,Hawai
1,2,"Sech,Daddy Yankee,J Balvin Featuring ROSALIA,F...",Relacion
2,3,"J Balvin,Dua Lipa,Bad Bunny & Tainy",Un Dia (One Day)
3,4,Ozuna,Caramelo
4,5,Nio Garcia x Anuel AA x Myke Towers x Brray x ...,La Jeepeta


In [4]:
# Normalize artist and title and drop duplicate rows
song_data['artists'] = song_data['artists'].str.upper() 
song_data['title'] = song_data['title'].str.upper() 
song_data.drop_duplicates(subset=['title', 'artists'], keep='first', inplace=True)

song_data.head()

,rank,artists,title
0,1,MALUMA,HAWAI
1,2,"SECH,DADDY YANKEE,J BALVIN FEATURING ROSALIA,F...",RELACION
2,3,"J BALVIN,DUA LIPA,BAD BUNNY & TAINY",UN DIA (ONE DAY)
3,4,OZUNA,CARAMELO
4,5,NIO GARCIA X ANUEL AA X MYKE TOWERS X BRRAY X ...,LA JEEPETA


# Get lyrics from MusicMatch api

In [11]:
# TODO: Ignorar las canciones que no tienen lyrics
# Referencia de como usar el api de music match https://youtu.be/WFRzKmpepj4
base_url= 'https://api.musixmatch.com/ws/1.1/'
api_key = "3950aca40989ee9f7d05723278e3fdfd"

In [12]:
def get_lyric(row):
    title = row['title']
    artist = row['artists']
    lyric_matcher = 'matcher.lyrics.get?format=json&callback=callback'
    url = f"{base_url}/{lyric_matcher}&q_track={title}&q_artist={artist}&apikey={api_key}"
    # print(title, artist, url)
    
    response = requests.get(url)
    data = response.json() # lee la información en json (cuidado con jsonp, se tiene que hacer mayor procesamiento)
    if data['message']['header']['status_code'] == 200:
        lyric = data['message']['body']['lyrics']['lyrics_body'].replace('******* This Lyrics is NOT for Commercial use *******', '')
        # print(lyric)
        
        row['lyric'] = lyric.replace('(1409620732275)',' ')
        row['explicit'] = data['message']['body']['lyrics']['explicit']
        row['lyric_provider'] = 'MusicMatch'
    return row

def get_music_genre(row):
    title = row['title']
    artist = row['artists']
    url = f"{base_url}/matcher.track.get?format=json&callback=callback&q_track={title}&q_artist={artist}&apikey={api_key}"
    # print(title, artist, url)
    
    response = requests.get(url)
    data = response.json()
    # print(data)
    if data['message']['header']['status_code'] == 200:
        track = data['message']['body']['track']
        # print(track)
        
        row['musixmatch_rating'] = track['track_rating']
        # row['artist_name'] = track['artist_name']
        # row['track_name'] = track['track_name']

        if len(track['primary_genres']['music_genre_list']) > 0:
            row['genre'] = track['primary_genres']['music_genre_list'][0]['music_genre']['music_genre_name_extended']
    return row

In [13]:
song_data = song_data.apply(get_lyric, axis=1)
song_data = song_data.apply(get_music_genre, axis=1)

# Get lyrics from Google api

In [5]:
def get_google_lyrics(row):
    # Simulate chrome, this prevent that google does not sent g-expandable-content elements
    headers = {
        # 'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
    # Generate google url
    title = row['title']
    artist = row['artists']
    page = requests.get(f"https://www.google.com/search?q={artist}-{title}", headers=headers)
    
    # if does not return response 200 exit
    if page.status_code != 200:
        print('Blocked: ', page.status_code)
        return row
    
    # Parse html
    html = BeautifulSoup(page.text, 'html.parser')
     
    # Find Lyrics google
    lyrics_select = html.select("div[data-lyricid] > g-expandable-content:nth-child(2)")#
    if len(lyrics_select) > 0:  # If google use LyricFind
        lyrics_container = lyrics_select[0]
        # print(title, artist, "Find Lyric")
        
        google_lyric = []
        for prose in lyrics_container.select("div"):
            for verse in prose.select("span"):
                google_lyric.append(verse.text + '\n')  # End of verse
            google_lyric.append('\n')  # End of prose
        google_lyric = ''.join(google_lyric) 

        row['lyric'] = google_lyric
        row['lyric_provider'] = 'Google'
    else:  # Todo: other Google lyrics provider
            pass  
    
    # Se agrega timer ramdom para que no se bloquee le IP, sinembargo se bloquea 
    time.sleep(random.uniform(1, 3))
    return row

In [6]:
song_data = song_data.apply(get_google_lyrics, axis=1)

HAWAI MALUMA Find Lyric
HAWAI MALUMA Find Lyric
UN DIA (ONE DAY) J BALVIN,DUA LIPA,BAD BUNNY & TAINY Find Lyric
CARAMELO OZUNA Find Lyric
AY,DIOS MIO! KAROL G Find Lyric
RITMO (BAD BOYS FOR LIFE) BLACK EYED PEAS X J BALVIN Find Lyric
DEL MAR OZUNA,DOJA CAT & SIA Find Lyric
EL ENVIDIOSO LOS DOS CARNALES Find Lyric
PORFA FEID,J BALVIN,MALUMA,NICKY JAM,SECH & JUSTIN QUILES Find Lyric
LA TOXICA FARRUKO Find Lyric
CON TUS BESOS ESLABON ARMADO Find Lyric
ADMV MALUMA Find Lyric
YAYA 6IX9INE Find Lyric
AQUI ABAJO CHRISTIAN NODAL Find Lyric
QUE MAL TE FUE NATTI NATASHA Find Lyric
TU CARIN LEON Find Lyric
BESAME PLAY N SKILLZ,DADDY YANKEE & ZION & LENNOX Find Lyric
CANCION CON YANDEL YANDEL X BAD BUNNY Find Lyric
PAM JUSTIN QUILES,DADDY YANKEE & EL ALFA Find Lyric
DIOSA MYKE TOWERS Find Lyric
DESPEINADA OZUNA X CAMILO Find Lyric
MENTE POSITIVA JUNIOR H Find Lyric
AZUL J BALVIN Find Lyric
VIDA VENTAJOSA LOS DOS CARNALES Find Lyric
REGGAETONERA ANUEL AA Find Lyric
MI CUARTO JERRY DI Find Lyric
REL

LO QUE ME ENAMORA DIEGO HERRERA Find Lyric
ESPEJEANDO KIKIN Y LOS ASTROS,LEANDRO RIOS Find Lyric
LA CASA EN VENTA LOS INVASORES DE NUEVO LEON Find Lyric
A LAS CUANTAS HORAS? LEANDRO RIOS Find Lyric
NO SOY COMO TU LOS CALIZ Find Lyric
DIME QUE NUMERO FUI PANCHO URESTI Find Lyric
CUANDO MENOS LO ESPERABA CRISTIAN JACOBO Find Lyric
LA MIRADA NUNCA MIENTE LOS 2 DE LA S Find Lyric
ES MAS FACIL LA GABBANA Find Lyric
COMO SI YO FUERA PASTEL LA ARROLLADORA BANDA EL LIMÓN Find Lyric
YA ME ENTERE CHAYIN RUBIO Find Lyric
AGUANTATE GABRIEL VIVAL Find Lyric
POR AMOR A TI SAUL EL JAGUAR Find Lyric
A PESAR DE TODO LUCERO Find Lyric
PARA QUE ME LAS DISTE GRUPO CODICIADO Find Lyric
CUPIDO FALLO PIPE BUENO Find Lyric
ENAMORADOS LOS INQUIETOS DEL NORTE Find Lyric
YA DILE LA VERDAD LOS HURACANES DEL NORTE Find Lyric
LA ESCUELA NO ME GUSTO ADRIEL FAVELA,JAVIER ROSAS Find Lyric
CERRANDO CICLOS BANDA MS Find Lyric
APARENTEMENTE BIEN JENNI RIVERA Find Lyric
EL PATROCINADOR IMPERIO CALAVERA Find Lyric
LLUVIA E

LA LOCA RAMONA PAOLO RUBBOLI Find Lyric
NO VAN A ENTENDER INTOCABLE Find Lyric
OJALA FANO GAXIOLA Find Lyric
LARGATE Y NO VUELVAS GRUPO CORPORACIÓN Find Lyric
DEFINITIVAMENTE DADDY YANKEE FT SECH Find Lyric
IMPERFECTAMENTE PERFECTA LORENZO MENDEZ Find Lyric
ANTÍDOTO LOS YAGUARU Find Lyric
NO ME ARREPIENTO MARCO ANTONIO SOLIS Find Lyric
TE VA A FALTAR MARILYN ODESSA Find Lyric
ALGO OFICIAL LA ORIGINAL BANDA EL LIMON Find Lyric
BOTA LA BATA BLANCO Y NEGRO Find Lyric
EL DESTINO CARLOS RIVERA,NATALIA JIMENEZ Find Lyric
TKN ROSALIA Find Lyric
UN CONSENTIDO DE DIOS LOS TIGRES DEL NORTE Find Lyric
CORAZONES INVENCIBLES SUPER LAMAS,ALEKS SYNTEK Find Lyric
FANTASIA OZUNA Find Lyric
SIN FORTUNA SANGRE FELINA Find Lyric
EL COLOR DE TUS OJOS BANDA MS Find Lyric
TODO LO ENCUENTRO EN TI SUPER LAMAS FT RAYMIX Find Lyric
DI BANDA LOS RECODITOS Find Lyric
PA OLVIDARME DE ELLA PISO 21,CHRISTIAN NODAL Find Lyric
LLORAR LOS SOCIOS DEL RITMO,ALEXANDER ACHA Find Lyric
EL QUEMADO BETO PEÑA Find Lyric
PROVOCA

PROBABLEMENTE CHRISTIAN NODAL Find Lyric
NO ME HUBIERA ENAMORADO CORNELIO VEGA Y SU DINASTIA Find Lyric
TUS JEFES NO ME QUIEREN GRUPO ENSAMBLE Find Lyric
QUE FALTA ME HACE MI PADRE ANTONIO AGUILAR Find Lyric
LA CARCACHA SELENA Find Lyric
ESPERO CON ANSIAS REMMY VALENZUELA Find Lyric
DONDE ESTARAS RAYMIX Find Lyric
DIME JULION ALVAREZ Find Lyric
SI TE FALTA ALGUIEN CHRISTIAN NODAL Find Lyric
RITMO J BALVIN Find Lyric
A QUIEN LE VA A DOLER LA EXPLOSIVA BANDA DE MAZA Find Lyric
CAGUATES PISTACHES BANDA MS Find Lyric
CORAZON MAGICO JULION ALVAREZ Find Lyric
SANTA CLAUS LLEGO A LA CIUDAD BANDA MS Find Lyric
YA NO ME PREGUNTES MIGUEL GALINDO Find Lyric
PEGALE PAPA LOS PLEBEYOS Find Lyric
AL CIEN Y PASADITO JORGE MEDINA Find Lyric
SOY DE RANCHO EL KOMANDER Find Lyric
LA REYNALDA NELSON KANZELA Find Lyric
YA NO VIVES EN MI LA BANDONONONA CLAVE NUEVA Find Lyric
DEL NEGOCIANTE LOS PLEBES DEL RANCHO DE ARIEL CAMACHO Find Lyric
Y ME BESA GERARDO ORTIZ Find Lyric
EN VIDA BANDA LOS SEBASTIANES Find 

SABES A CHOCOLATE KUMBIA KINGS Find Lyric
ARBOLES DE LA BARRANCA EL COYOTE Y SU BANDA Find Lyric
GENTE BATALLOSA BANDA CARNAVAL Find Lyric
LE HACE FALTA UN BESO EL CHAPO DE SINALOA Find Lyric
LOCO (EN VIVO) PESADO,CELSO PIÑA Find Lyric
SENTIMIENTOS ALICIA VILLAREAL,MARIA JOSE Find Lyric
COMO TU MUJER LUZ MARIA Find Lyric
ME HUBIERAS DICHO JOSS FAVELA Find Lyric
LA NOCHE QUE CHICAGO MURIO BANDA TORO Find Lyric
ORO BRONCO Find Lyric
TE DIJE JOSS FAVELA Find Lyric
LA VIDA ES UN CARNAVAL CELIA CRUZ Find Lyric
LOS CAMINOS DE LA VIDA LA TROPA VALLENATOS Find Lyric
MIL CADENAS LOS RIELEROS DEL NORTE Find Lyric
17 AÑOS LOS ANGELES AZULES FT. JAY DE LA CUEVA Find Lyric
LA NOCHE GRUPO CAÑAVERAL DE HUMBERTO PABON Find Lyric
UN FIN EN CULIACAN (PERRON) LA ADICTIVA BANDA SAN JOSE DE MESILLAS Find Lyric
NO ME LLAMES MAS MEXIKOLOMBIA Find Lyric
CARICIAS CLANDESTINAS REMMY VALENZUELA Find Lyric
QUIEN ES USTED SERGIO VEGA Find Lyric
LO PRIMERO QUE HARIA ALEX FERNANDEZ Find Lyric
AUNQUE AHORA ESTES CON 

ENHORABUENA FIDEL RUEDA Find Lyric
EL PIROPO PEPE GOMEZ JR Y SU GRUPO UNION 82 Find Lyric
HASTA QUE SALGA EL SOL OZUNA Find Lyric
TE QUIERO,TE AMO PESADO Find Lyric
SENTIMIENTOS DE CARTON DUELO Find Lyric
NIÑA DE MI CORAZON LA ARROLLADORA BANDA EL LIMÓN Find Lyric
TU YA ERES COSA DEL PASADO FIDEL RUEDA Find Lyric
NECESITO DUEÑA SERGIO VEGA Find Lyric
NI QUE ESTUBIERAS TAN BUENA CALIBRE 50 Find Lyric
LLORAR LOS SOCIOS DEL RITMO Find Lyric
SOY FELIZ FIDEL RUEDA Find Lyric
SOY ASI VALENTIN ELIZALDE Find Lyric
LA GRAN SEÑORA JENNI RIVERA Find Lyric
VENCEDOR VALENTIN ELIZALDE Find Lyric
HASTA QUE UN DIA GRUPO FIRME Find Lyric
EL AMIGO QUE SE FUE INTOCABLE Find Lyric
FELICES LOS 4 (SALSA VERSION) MALUMA,MARC ANTHONY Find Lyric
NO VOY A DETENERTE PESADO Find Lyric
ME LLAMO RAQUEL BANDA MACHOS Find Lyric
TE DIGO VETE LOS ASKIS Find Lyric
SERIA UN ERROR REGULO CARO Find Lyric
HUELE A PELIGRO LA ARROLLADORA BANDA EL LIMÓN Find Lyric
LA MONEDA VAGON CHICANO Find Lyric
A DONDE QUIERA QUE VAYA JULI

EL CHUBASCO CARLOS Y JOSE Find Lyric
SIN RESPIRACION BANDA EL RECODO Find Lyric
EL PERRO,EL GATO Y YO LOS GUARDIANES DEL AMOR Find Lyric
ME GUSTA VIVIR DE NOCHE LOS TUCANES DE TIJUANA Find Lyric
LO HUBIERAS HECHO CON TU CORAZON BANDA TIERRA MOJADA Find Lyric
LA TUMBA FALSA LOS TIGRES DEL NORTE Find Lyric
MAS CARO QUE AYER GERARDO ORTIZ Find Lyric
EL GRAN BARON SONORA TROPICANA Find Lyric
30 CARTAS REMMY VALENZUELA Find Lyric
AHORA ESTOY EN TU LUGAR ALTO NIVEL Find Lyric
LOCO PESADO Find Lyric
ME PROMETI OLVIDARTE BANDA EL RECODO Find Lyric
NO TE VAYAS INTOCABLE Find Lyric
LO QUE TE AMO CRECER GERMAN Find Lyric
PERO ME PERDON EL PODER DEL NORTE Find Lyric
EL NOA NOA JUAN GABRIEL Find Lyric
ESE LOS CARDENALES DE NUEVO LEON Find Lyric
VOY A PISTEARME EL DOLOR REGULO CARO Find Lyric
EL BURRITO DE BELEN DUELO Find Lyric
DE UNO Y DE TODOS LOS MODOS PALOMO Find Lyric
SIGNO LIBRA GRUPO LIBRA Find Lyric
FUISTE MALA KUMBIA KINGS Find Lyric
UN MILLON DE PRIMAVERAS VICENTE FERNANDEZ Find Lyric
MUN

VOLVERE A AMAR VALENTIN ELIZALDE Find Lyric
ME GUSTAS JOAN SEBASTIAN Find Lyric
LAPIZ LABIAL LOS FELINOS Find Lyric
PIDEME PERDON BANDA MS Find Lyric
MI BUENA SUERTE LOS TIGRES DEL NORTE Find Lyric
ME DUELE ROBERTO TAPIA Find Lyric
POR TU AMOR ALACRANES MUSICAL Find Lyric
DOS LOCOS LOS HOROSCOPOS DE DURANGO Find Lyric
CON LA PALMA DE MI MANO BANDA SAN JOSE DE MESILLAS Find Lyric
Y POR QUE NO ALFREDO OLIVAS Find Lyric
ELLA ES MI MUJER BANDA CARNAVAL Find Lyric
DOS CARTAS Y UNA FLOR LOS CAMINANTES Find Lyric
LA CANELERA LOS GFEZ Find Lyric
LA HIERBA SE MOVIA TROPICALISIMO APACHE Find Lyric
COLEGIALA MARGARITA Find Lyric
GRACIAS POR TU AMOR BANDA EL RECODO Find Lyric
LA NIÑA JULION ALVAREZ Find Lyric
SI TU AMOR NO VUELVE LA ARROLLADORA BANDA EL LIMÓN Find Lyric
BUENOS DIAS SEÑOR SOL JUAN GABRIEL Find Lyric
SUENA LA BANDA LOS TUCANES DE TIJUANA,CODIGO FN Find Lyric
PREGUNTALE EDWIN LUNA Y LA TRAKALOSA DE MONTERREY Find Lyric
EL PRIMER TONTO LOS CARDENALES DE NUEVO LEON Find Lyric
EL CAMPES

In [15]:
song_data.head(50)

,artists,explicit,genre,lyric,lyric_provider,musixmatch_rating,rank,title
0,MALUMA,NaN,NaN,Deja de mentirte (ah)\nLa foto que subiste con...,Google,NaN,1,HAWAI
1,"SECH,DADDY YANKEE,J BALVIN FEATURING ROSALIA,F...",1.0,NaN,NaN,MusicMatch,90.0,2,RELACION
2,"J BALVIN,DUA LIPA,BAD BUNNY & TAINY",NaN,NaN,You know that sometimes\nI think about us now ...,Google,NaN,3,UN DIA (ONE DAY)
3,OZUNA,0.0,Latin / Pop in Spanish,"Uoo oh (oye ma')\nBaby, baby\nOzuna\n\nAunque ...",MusicMatch,79.0,4,CARAMELO
4,NIO GARCIA X ANUEL AA X MYKE TOWERS X BRRAY X ...,NaN,NaN,NaN,NaN,NaN,5,LA JEEPETA
5,KAROL G,0.0,NaN,"Ovy on the drums\n\nYo no te estaba buscando, ...",MusicMatch,80.0,6,"AY,DIOS MIO!"
6,BLACK EYED PEAS X J BALVIN,1.0,Latin / Pop in Spanish,"This is the rhythm, rhythm, rhythm, rhythm\nTh...",MusicMatch,82.0,8,RITMO (BAD BOYS FOR LIFE)
7,TAINY X J BALVIN,0.0,Music,NaN,MusicMatch,80.0,9,AGUA
8,"OZUNA,DOJA CAT & SIA",1.0,NaN,Ah-ah-ah-oh-oh-oh\nAh-ah-ah-yo-oh (yeah-eh)\nA...,MusicMatch,81.0,10,DEL MAR
9,"BLACK EYED PEAS,OZUNA + J.REY SOUL",1.0,NaN,NaN,MusicMatch,26.0,11,MAMACITA


# Export lyrics data

In [14]:
song_data.to_csv(
    top_songs_filename, 
    index=False, 
    quoting=csv.QUOTE_NONNUMERIC  # Prevent errors on strings with , and \n characters
)